- # 작사가 인공지능

이번엔 조금 특이하게 작사가 인공지능을 만들었다.  
input data를 통해서 어떤 객체인지 판별하는 분류기가 아닌 새로운 산출물을 만들어내는  
토이 프로젝트였기 때문에 조금 새로운 느낌이 들었다.

- ## 데이터 준비  
하지만 역시나 그렇듯 input data는 존재한다. 파일을 변수에 담아준다.

In [1]:
import glob
import os
import re
import tensorflow as tf
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Now greetings to the world! Standing at this liquor store,', 'Whiskey coming through my pores,', 'Feeling like I run this whole block.']


담아준 텍스트의 일정 부분을 출력해 보았다.

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0 or len(sentence) > 15: continue   # 길이가 0인 문장은 skip
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장 skip
    if idx > 9: break   # 문장 10개만 확인
        
    print(sentence)

enumerate 함수를 통해 raw_corpus에 들어있는 텍스트의 기본적인 전처리를 해준다.  


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바꾼다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


이번 단계에서는 문장 전처리에 대한 함수를 정의했다. 위의 전처리 단계에서는 먼저 체로  
학습할 데이터 들을 
<strong>걸러 내는 듯한 느낌</strong>이었다면 이번 전처리에서는 데이터를 <strong>다루기 쉽게 만들어주는 단계</strong>이다.

In [4]:
corpus = []
for sentence in raw_corpus: 
    if len(sentence) == 0 or len(sentence) > 15: continue #문장의 길이가 0이거나
                                                          #15 이상이라면 skip
    if sentence[-1] == ":":continue #문장의 끝이 :이라면 skip
        
    corpus.append(preprocess_sentence(sentence))#corpus에 전처리된 문장을 추가
    
print(sentence)

Like a serial killer, man is a goner (man cured) 


## Tokenizing  
--------------------------------

- ### 토큰화란?  
코퍼스 데이터(말뭉치 - 사람들이 사용하는 자연어로 보면 된다!)가 필요에 맞게 전처리되지  
않았을 때 입력데이터로 활용하고자 한다면 반드시 토큰화가 필요한데, token은 보통 단어나  
어떤 부분이 의미를 가질 때 정해진다.  

아래 코드는 토큰화를 해주는 코드이다.

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    #전처리 로직을 따로 설정해주지 않을 것(커스터마이징이 가능하다)
        oov_token="<unk>"  # <unk> == unknown 사전에 없는 단어는 해당 토큰으로 대체된다.
    )
    tokenizer.fit_on_texts(corpus)   #fit_on_text 함수를 통해서 사전을 자동구축할 수 있다.

   #tokenizer를 통해서 입력할 수 있는 데이터로 만들어준다.
    tensor = tokenizer.texts_to_sequences(corpus)   #text_to_sequeces는 corpus -> tensor의 역할을 한다.
    # 입력 데이터의 시퀀스 길이를 맞춰주기 위해 padding 함수를 사용할 수 있다.
    # maxlen의 디폴트값은 None인 경우 가장 긴 문장을 기준으로 시퀀스의 길이를 맞춰준다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  

    print(tensor,tokenizer)
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    8   14 ...    0    0    0]
 [   2    8   14 ...    0    0    0]
 [   2    8   14 ...    0    0    0]
 ...
 [   2   72   26 ...    0    0    0]
 [   2  140    3 ...    0    0    0]
 [   2   78 3416 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f608312be90>


## fit_on_text  
주석으로는 '사전을 구축 한다'라는 바로 와닿지 않는 말을 사용했는데,  
사실 이 함수는 문자 데이터를 입력받아서 리스트의 형태로 변환해준다. 

In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성

print(src_input[0])
print(tgt_input[0])

[ 2  8 14 14  3  0  0  0  0  0  0  0  0  0]
[ 8 14 14  3  0  0  0  0  0  0  0  0  0  0]


위 src_input은 마지막 토큰을 잘라내었기 때문에 < end >로 끝날 수 있다고 생각하기  
쉽지만 위에서 padding 작업을 했기 때문에 < pad >로 끝날 가능성이 더 높다 ! 

In [7]:
BUFFER_SIZE = len(src_input) #buffer는 입력할 문장의 개수로 해주는 것이 당연!
BATCH_SIZE = 256 
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 
                                        #<pad>를 포함하여 7001개
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## from_tensor_slices()??  
----------------------------------------  
tf.data.Dataset에서의 from_tensor_slices()는 <strong><span style = "color  : blue">주어진 입력 텐서의 첫번째 차원을 따라 슬라이스</span></strong> 해준다. 이때 모든 입력 텐서들은 첫번째 차원과 같은 크기를 가져야한다고 한다!

이제 직접적으로 텍스트를 만들어줄 text Generator 클래스(모델)를 만들어준다

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size): #초기화 함수에
                                                                #각각 레이어를 달아준다
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #워드 벡터의 차원수(단어가 추상적으로 표현되는 크기. 값이 커질수록
                    #단어의 추상적인 특징들을 더 잘 잡아내는 반면 이에 맞는 데이터가 주어
                    #지지 않는다면 오히려 그 특징들을 잡아낼 수 없을 것이다.)
hidden_size = 1024
#모델에 얼마나 많은 손을 달아줄 것인가? embedding_size와 비슷하게 크기에 맞는 데이터가
#주어지지 않는다면 허공에 손을 휘젓는 꼴이 될 것이다
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [9]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 7001), dtype=float32, numpy=
array([[[ 9.56580916e-05, -3.37890058e-04,  1.29056207e-05, ...,
         -1.50304171e-04, -1.53379762e-04,  4.73160026e-06],
        [ 6.88346845e-05, -8.85950343e-04,  2.21441122e-04, ...,
         -6.60828431e-04, -3.76987387e-04,  1.04010804e-04],
        [ 1.49475760e-04, -1.18420378e-03,  2.00043447e-04, ...,
         -6.30448456e-04, -3.13722121e-04,  4.13638336e-05],
        ...,
        [-5.85275178e-04, -1.01008639e-03,  9.90796951e-04, ...,
          2.05876119e-03,  8.42770794e-04,  3.85059649e-03],
        [-7.00698583e-04, -1.03843352e-03,  1.09440170e-03, ...,
          2.15463340e-03,  9.07822687e-04,  3.93811474e-03],
        [-8.02449125e-04, -1.07970694e-03,  1.18517282e-03, ...,
          2.20872555e-03,  9.57608398e-04,  3.96862207e-03]],

       [[ 9.56580916e-05, -3.37890058e-04,  1.29056207e-05, ...,
         -1.50304171e-04, -1.53379762e-04,  4.73160026e-06],
        [ 3.07370647e-04, -4.52753011e-04, -3.

위의 코드로 나온 텐서의 shape는 256, 14, 7001의 값을 갖는다.  
256은 위에서 정해준 batch size, 14는 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미를 가지고, 7001은 익숙하게도 pad를 포함한 단어사전의 크기이다.

## 모델 확인  
모델 구성을 완료했기 때문에 원하는 형태로 모델이 만들어졌는지 확인해 볼 수 있다.

In [10]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


## 학습

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
55/55 [==============================] - 95s 2s/step - loss: 2.3591
Epoch 2/30
55/55 [==============================] - 106s 2s/step - loss: 1.3731
Epoch 3/30
34/55 [=================>............] - ETA: 53s - loss: 1.2398

## 텍스트 생성
학습까지 완료된 모델을 통해서 텍스트를 만들어줄 수 있다. 

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 우선 입력받은 단어를 텐서로 변환해준다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 루프를 돌면서 단어 하나씩 생성. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]  
 
        test_tensor = tf.concat([test_tensor, 
                                tf.expand_dims(predict_word, axis=0)], axis=-1)

        #<end>를 예측하거나 maxlen에 다다를때 까지 루프 반복
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 문장 반환

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i")

## 회고  
새로운 산출물을 내는 모델은 새로웠다. 사실 예측이라는 측면에서 분류기 모델들과  
크게 다르지 않다는 생각이 들었고, 너무 복잡하게 생각할 필요도 없다는 생각이 들었다.  
언어학을 전공해서 크게 어렵지 않겠다는 생각을 가지고 시작한 토이프로젝트 였지만, 사실  
코퍼스나 진폭, 주파수(음성 분석) 등 용어만 익숙하고 개념 자체는 오히려 더 어색했다.  
해당 프로젝트를 진행하면서 모델이 어떻게 문법을 예측하는지, 적절한 단어를 고르는지에 대한  
모든 인사이트는 들어오지 않았지만, 어렴풋이 감을 잡을 수 있는 토이프로젝트 였던 것 같다.